# Template für K-means Clustering | Multi-Node/Multi-GPU


***

<img src="./pictures/mog.PNG"  width="1000px;" hight="900px;">

In diesem Abschnitt wird erkläre, wie man den KMeans Algorithmus auf Worker skaliert und zum Laufen bringt. Dazu nutzen wir Dask und Rapids. Dask bietet uns ein Dashboard wo wir Einzelheiten verfolgen können. Dask-Cuda lässt sich gut mit PyTorch und anderen Frameworks kombinieren, ohne die Clustereinstellungen groß zu ändern.

In dem Nootebook für das Setup wird erklärt, wie man selber sowas auf seinem eigenen PC/Workstation/Cluster installiert. Es gibt auch andere Möglichkeiten das zu installieren. Hier nutzen wir ein Cluster Setup mit Nodes, installiert mittels Docker.   

<u>Hinweis:</u><br>
Wenn es mal hängt, muss ggf. der Cluster, Worker oder das Notebook neu gestartet werden. Sowas passiert, wenn z.B. eine Exception vorkommt.   

<br>

Dask: https://www.dask.org | https://docs.dask.org/en/stable/ <br>
Rapids: https://rapids.ai <br> 

# 1. Rapids und Dask

<img src="./pictures/raspids_oss.PNG"  width="825px;" hight="725px;">
(Bild: (https://on-demand.gputechconf.com/gtcdc/2018/pdf/dc8256-rapids-the-platform-inside-and-out.pdf))

Rapids bietet uns eine Sammlung von Libraries, die uns bei vielen Dingen unterstützen kann. Eine dieser Libraries, die wir nutzen werden, nennt sich cuDF. cuDF sind Cuda Dataframes, die erstmal nur auf einer GPU laufen. Die Idee dahinter ist, dass wir <u>nah zu dasselbe machen</u> wie bei Pandas (Panda DataFrame), aber nur auf einer GPU. Dadurch können viele Berechnungen viel schneller durchgeführt werden. Beispielsweise das Vorverarbeiten von Daten vor dem Trainieren, oder anderweitige Zwecke. Die API ist nah zu identisch wie bei Pandas. 

Rapids funktioniert <u>nur</u> mit <u>Nvidia Grafikkarten</u>.

Rapids ist OpenSource, die API Beschreibungen der Libraries verlinken den dahinterliegenden Code. Der Code enthählt selber nochmal Kommentare und erklärungen.

Rapids entwickelt sich und es kommen Updates sowie Erweiterungen. Algorithmen, die noch nicht (Single-/Multi-Node/ GPU) verfügbar sind, können noch in den neueren Versionen auftauchen. Das gilt besonders für Multi-Node/Multi-GPU Implementierungen der Algorithmen. Es kann sein das manche Multi-Node/Multi-GPU Algorithmen sich am Ende nicht zusammenfassen lassen können, oder das Speichern/Laden sowie die Verwendung dieser auf GPU/CPU eingeschränkt sind. Andere Multi-Node/Multi-GPU Algorithmen können so wie in diesem Template hier verwendet werden. Einge andere können sich in der Anwendung unterscheiden. 


Floating-Point Berechnungen mit cuDF werden auf der GPU parallel ausgeführt, also ist die Reihenfolge nicht deterministisch. Die Ergebnisse können von Pandas abweichen.
a + b ist nicht gleich b + a. Das kann z.B. bei data.sum() vorkommen.
cuDF unterstützt auch verschiedene Datentypen wie int32, int64, float32, float64. Die ganze Liste ist hier zu finden.

Es gibt noch Dask-cuDF, diese nutzt man unter anderem, um große Daten zu Partitionieren. Die API ist Dask-ähnlich. 

Eine weitere Library von Rapids ist cuML, welche auch cuDF nutzt. Diese enthält verschiedene supervised und unsupervsed Algorithmen. Der Fokus hier liegt dabei, dass die Algorithmen möglichst auf einer GPU laufen. Ein Teil dieser Algorithmen kann mittels eines Clusters (cuML nutzt dafür ein Dask Cluster) auf verschiedene Nodes und GPUs skalieren. 

Die API von cuML ist fast genauso aufgebaut wie bei 
Sklearn. <br> Die Release Notes von cuML sind hier zu finden (https://github.com/rapidsai/cuml/releases).
  

Was wir neben Rapids und Dask noch nutzen können ist CuPy. CuPy ist wie Numpy, nur läuft es unter auf GPU. CuPy wird hier nicht detailliert erklärt.

<br> 

cuDF Cuda DataFrames Dok.:  https://docs.rapids.ai/api/cudf/stable/ <br>
cuDF API: https://docs.rapids.ai/api/cudf/stable/api_docs/api/cudf.dataframe/ <br>
cuDF Floating-Point Berechnung: https://docs.rapids.ai/api/cudf/stable/user_guide/pandas-comparison/#floating-point-computation        <br>
cuDF Datentypen: https://docs.rapids.ai/api/cudf/stable/user_guide/data-types/ | https://docs.rapids.ai/api/cudf/stable/user_guide/io/ <br>

cuML GitHub: https://github.com/rapidsai/cuml  <br>
cuML Dok.: https://docs.rapids.ai/api/cuml/stable/ <br>

Dask-cuDF: https://docs.rapids.ai/api/dask-cudf/stable/

CuPy: https://cupy.dev



Was wir nutzen:<br>
cuDF: DataFrames deren API nahezu identisch mit Pandas ist. Also quasi Pandas auf GPU.<br>
dask_cudf: Dask bietet uns DataFrames, die verteilt sind. Mit dask_cudf läuft das ganze auf GPU. Also quasi Dask auf GPU. <br>
cupy: wie Numpy und Scipy aber auf GPU.

<img src="./pictures/df.PNG"  width="725px;" hight="725px;">

Wir teilen das Training in 4 Bereiche auf. Hier nutzen wir MNSG.

<img src="./pictures/rapids_flynn.PNG"  width="725px;" hight="725px;">

Das Bild zeigt in welchen Bereichen man welche DataFrames nutzen kann. Pandas liegt dabei nicht im Fokus.

<img src="./pictures/zuordnung.PNG"  width="1050px;" hight="1050px;">

Da wir mit Dask-cuDF und Dask große Daten lesen und partitionieren können, ist auch der Einsatz im SNSG Bereich möglich. cuDF unterstützt das lesen mehrere Dateien (bis jetzt)  nicht, so das man Dask oder Dask-cuDF nutzen muss, wenn man globstings nutzt, um alle CVS Dateien in einem Verzeichnis zu lesen.

Es gibt auch die Möglichkeit cuDF als Backend für Dask zu nutzen.

<br>

cuDF Backend für Dask: https://docs.rapids.ai/api/dask-cudf/stable/#dataframe-creation-from-on-disk-formats

<b>[Bild Konvertierungen]</b>

<u>Hinweis:</u><br>
Variablen die GPU Speicher nutzen und nicht mehr gebraucht werden, sollten mittels `del` gelöscht werden. Wenn nach dem Löschen immer noch Speicher belegt ist, muss der Kernel des Notebooks neugestartet werden. 

In [6]:
# Imports 
import cupy as   cp
import cudf
import cuml
from cuml.dask.common import utils as dask_utils
import dask_cudf
import dask

import numpy as np
from numpy import genfromtxt

import pandas as pd
import os
import time

Da dier API wie Pandas aufgebaut ist, wird cuDF im detail nicht erklärt. Bei interesse kann man sich die API durchlesen (https://docs.rapids.ai/api/cudf/stable/user_guide/pandas-comparison/).  

In [2]:
# cuDF und Panda series 
s_cuda = cudf.Series([1, 2, cudf.NA])
s_panda = pd.Series([1, 2, pd.NA])

print(f"Pandas:\n{s_cuda}\n")
print(f"Rapids:\n{s_panda}")

Pandas:
0       1
1       2
2    <NA>
dtype: int64

Rapids:
0       1
1       2
2    <NA>
dtype: object


## 1.1 Start Dask-Cluster (Mit Docker)

Pro GPU wird genau ein Prozess für einen Worker gestartet. Die Worker-ID wird hochgezählt. <br>

<u>Hinweise:</u><br>
Die MNMG Versionen der Algorithmen können auch auf SNSG laufen.

<img src="./pictures/rapids_cluster2.PNG">

In [1]:
# Ein Modul, das uns unterstützt.
import rapids_tools as rapids_tools    

Rapids-tools 1.0
Umgebungsvariablen werden gesetzt
NCCL_SOCKET_NTHREADS: 4
NCCL_NSOCKS_PERTHREAD: 2


In [3]:
## Erstelle Client 
# rapids_tools wird mit dem Cluster geteilt, um Umgebungsvariablen zu setzen.
client = rapids_tools.create_dask_client()  # Gebe IP an. Standard:  127.0.0.1:8786 (Scheduler ist da wo auch Jupyter läuft)
client

Client IP: 127.0.0.1:8786


<Client: 'tcp://149.201.182.203:8786' processes=3 threads=3, memory=187.58 GiB>

Wenn wir eigene Python Module erstellen, können wir diese auch mit dem Cluster teilen.

Beim Ausführen sollte der Status "OK" ausgegeben werden. Wenn die Worker oder der Cluster neugestartet werden, müssen die Module ggf. neu geteilt werden.

In [4]:
# Lade Datei hoch. Gibt Status aus... 
client.upload_file('rapids_tools.py')  # Nur als Beispiel

{'tcp://149.201.182.188:37547': {'status': 'OK'},
 'tcp://149.201.182.203:36965': {'status': 'OK'},
 'tcp://149.201.182.205:46201': {'status': 'OK'}}

In [5]:
# Zeige Schlüssel. Die Ports ändern sich. 
client.has_what().keys()

dict_keys(['tcp://149.201.182.188:37547', 'tcp://149.201.182.203:36965', 'tcp://149.201.182.205:46201'])

In [175]:
# Starte alle Worker neu
client.restart_workers( client.has_what().keys() )
client.has_what().keys()

dict_keys(['tcp://149.201.182.188:37547', 'tcp://149.201.182.203:36965', 'tcp://149.201.182.205:46201'])

In [8]:
# Startet client neu.
#client.restart()

# 2. Daten

## 2.1 Daten generieren

Als Anfang erstellen wir Daten. Daten die auf dem Host sind, müssen erst verteilt werden.

cuML bietet uns Möglichkeiten Datensets zu erstellen.

In [7]:
# Imports 
from cuml import datasets            # Um Beispiele auf dem Host zu generieren
from cuml.model_selection import train_test_split

Ein Weg an Daten zu kommen, ist Datensets zu nutzen, die schon mit den Frameworks kommen. 

Das können wir nutzen, um den Aufbau des Codes zu testen und zu überprüfen, ob die Installation von Rapids und die Dask-Worker funktionieren.

Es gibt einige Parameter, die wir eingeben können, um Daten so zu generieren wie wir es wollen. Um es einfach zu halten, nutzen wir nur 2 Parameter.

<br>

Generiere Single-GPU Daten: https://docs.rapids.ai/api/cuml/stable/api/#dataset-generation-single-gpu

In [8]:
# Trainings- und Testgröße
train_size = 2000
test_size = 100
n_samples = train_size + test_size

# Erstelle X und y Daten für ein Cluster Problem 
X, y = datasets.make_blobs(n_samples=n_samples, n_features=2)
X, X_test, y, y_test = train_test_split( X, y, train_size=0.70, random_state=5)

Jetzt können wir uns anschauen was gemacht wurde, so könnten unsere Trainingsdaten auch aussehen.

In [9]:
print(f"X: {type(X)} y: {type(y)} \n X[0]: {X[0]}  \n y[0]: {y[0]} \n Länge X, y: {len(X)}, {len(y)}")

X: <class 'cupy.ndarray'> y: <class 'cupy.ndarray'> 
 X[0]: [ 7.4640355 -5.0711775]  
 y[0]: 0.0 
 Länge X, y: 1470, 1470


Die erstellten Daten sind in CuPy Format. Für das Trainieren müssen die Daten im Dask-cuDF Format sein. Alternativ können die Daten im Format Dask Array sein, mit CuPy als Backend (Siehe fit() https://docs.rapids.ai/api/cuml/stable/api/#id43).

## 2.2 Dask und Dask_cuDF

Schauen wir uns den Fall an, das die Daten <u>erst verteilt werden</u> müssen. 

In [12]:
# X ist auf dem Host und nicht Partitioniert.
X

array([[-0.29647613, -7.2054286 ],
       [-3.6063309 ,  8.981853  ],
       [ 3.1586378 , -7.306879  ],
       ...,
       [-3.3382192 , -7.088197  ],
       [-4.296981  ,  9.369374  ],
       [ 3.2598648 , -8.497779  ]], dtype=float32)

Bei Verteilen oder Laden der Daten mit HDFS, können wir eine Chunkgröße angeben oder die Anzahl der Partitionen. Die Worker übernehmen eine Anzahl der Partitionen für das Trainieren.

X ist ein cupy. Wir brauchen ein dask_cuDF oder ein Dask Array mit cupy als Backend. 

Erst wandeln wir das Array X in ein DataFrame um. Dann werden wir X partitionieren.

In [10]:
# cupy -> cuDF
X_cudf = cudf.DataFrame(X)
y_cudf = cudf.Series(y)
print(f"X Type: {type(X_cudf)} \nX ist:\n {X_cudf}\ny Type: {type(y_cudf)}\ny ist:\n{y_cudf}")

X Type: <class 'cudf.core.dataframe.DataFrame'> 
X ist:
 [[  7.4640355  -5.0711775]
 [-10.350191    8.24481  ]
 [  8.462273   -5.0934644]
 ...
 [  8.086229   -7.161651 ]
 [  7.671333   -6.8324304]
 [  9.996122   -6.2191343]]
y Type: <class 'cudf.core.series.Series'>
y ist:
0       0.0
1       1.0
2       0.0
3       0.0
4       2.0
       ... 
1465    2.0
1466    1.0
1467    0.0
1468    0.0
1469    0.0
Length: 1470, dtype: float32


Danach werden wir das DataFrame Partitionieren.

In [11]:
workers = client.has_what().keys()
n_workers = len(workers)
print(f"nworker: {n_workers}")

nworker: 3


In [12]:
# cuDF -> dask_cuDF
X_dask = dask_cudf.from_cudf( X_cudf, npartitions=n_workers )  # oder  npartitions = n_workers
y_dask = dask_cudf.from_cudf( y_cudf, npartitions=n_workers )

X_dask, y_dask = dask_utils.persist_across_workers(client, [X_dask, y_dask], workers=client.has_what().keys())  # Halte Ergebnis im Speicher 

Wir haben jetzt ein dask_cudf das partitioniert ist. Es gibt n partitionen die auf die Worker verteilt sind. 

In [13]:
print(f"dask_X: {type(X_dask)}\n{X_dask}")

dask_X: <class 'dask_cudf.core.DataFrame'>
<dask_cudf.DataFrame | 3 tasks | 3 npartitions>


In [14]:
X_dask

,0,1
npartitions=3,,
0,float32,float32
490,...,...
980,...,...
1469,...,...


In [15]:
## Damit sehen wir was der Cluster hat.
# Jeder hat ein Teil des DataFrames. Jeder Worker nutzt die eigenen Partitionen. 
client.has_what()

{'tcp://149.201.182.188:37547': ("('from_cudf-a1045aa6d093416772b18f93e41ccb8c', 1)",
  "('from_cudf-f178b64fefe6219d5cd1ffd825e66170', 1)"),
 'tcp://149.201.182.203:36965': ("('from_cudf-a1045aa6d093416772b18f93e41ccb8c', 0)",
  "('from_cudf-f178b64fefe6219d5cd1ffd825e66170', 0)"),
 'tcp://149.201.182.205:46201': ("('from_cudf-a1045aa6d093416772b18f93e41ccb8c', 2)",
  "('from_cudf-f178b64fefe6219d5cd1ffd825e66170', 2)")}

So siet der Flow aus, wenn wir Daten als Host erzeugen und dann verteilen.

<img src="./pictures/part.PNG"  width="925px;" hight="925px;">

Zusammeengefasst: <br>
Wenn wir Daten als Host erstellen (z.B. um den Aufbau zu testen), müssen die Daten Partitioniert werden. Das verteilen geht gut mit <u>geringen Datenmengen</u>.

Nach dem Verteilen hat jeder Worker ein Teil dieser Daten. Diese Teile sind Partitionen.

<img src="./pictures/part2.PNG"  width="925px;" hight="925px;">

## 2.3 Daten sind schon Lokal verfügbar 

Mit Dask-cuDF können wir Daten direkt in die GPU laden und Daten schreiben. Es gibt viele Parameter, die angegeben werden können (https://docs.rapids.ai/api/dask-cudf/stable/api/#creating-and-storing-dataframes). 

Wenn die Daten nicht in den Speicher passen, können diese partitioniert werden, indem wir die Daten mit Dask-cuDF laden.

<img src="./pictures/rapids_chunk.PNG">

Wir können Daten mit Dask laden und daraus ein Dask-cuDF erstellen.

Intern nutzt jeder Worker cuDF um die Daten zu lesen. Es können verschiedene Quellen angegeben werden wie HDFs, S3, http, ... (https://docs.dask.org/en/stable/how-to/connect-to-remote-data.html)

Wir können auch direkt Dask-cuDF nutzen um Daten zu laden. Das geht genau so wie bei Dask. Das schreiben der Daten mit Dask-cuDF folg auch der gleichen API.

Jeder Worker sollte die gleichen Pfade sehen können.

Das erstellte Dask-cuDF kann dann zum trainieren genutzt werden. 

<br>

Rapids Dask-cuDF Creating and storing DataFrames:  https://docs.rapids.ai/api/dask-cudf/stable/api/ <br>
Reading Larger than Memory CSVs with RAPIDS and Dask: https://medium.com/rapids-ai/reading-larger-than-memory-csvs-with-rapids-and-dask-e6e27dfa6c0f


<u>Hinweis zu HDFS:</u><br>
Das lesen von vielen Dateien oder einer großen Datei bringt keinen Vorteile der Datenlokalität mehr, siehe (https://stackoverflow.com/questions/54565738/does-dask-communicate-with-hdfs-to-optimize-for-data-locality)<br>
Das kann sich noch ändern. 
 


# 3. Multi-Node/Multi-GPU Training: Dask und Dask_cuDF

Wie bereits erwähnt, ist die API fast genau so wie bei Sklearn. Wenn wir jetzt Daten haben und diese noch verarbeiten wollen, muss darauf geachtet werden, dass die Datentypen am Ende richtig sind. 


Für das Model gibt es einige Parameter, die wir einstellen können, dieser Auszug zeigt einige Parameter: <br>
<i>n_clusters=8, max_iter=300, random_state=1, tol=1e-4, ...</i>
- n_clusters: The number of centroids or clusters you want.
- max_iter: The more iterations of EM, the more accurate, but slower.
- tol: Stopping criterion when centroid means do not change much.


Beim Trainieren werden nur die Mittelpunkte nach jeder Iteration der Cluster ausgetauscht. Jeder Worker bekommt eine Kopie des Models. <br>

<br>

Sklearn Kmeans: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html <br>
cuML MNMG Kmeans: https://docs.rapids.ai/api/cuml/stable/api/#id43


In [16]:
# Imortiere MNMG Kmeans
from cuml.dask.cluster.kmeans import KMeans as multi_kmeans

Jetzt kann man wie üblich vorgehen. Das Model Trainieren, Speichern, Laden und Evaluieren.

Wir können das Dashboard von Dask öffnen und sehen unter GPU die Auslastung (wenn es nicht zu schnell fertig ist).

In [17]:
multi_node_model = multi_kmeans(init="k-means||", random_state=100)      

%time multi_node_model.fit(X_dask)  

CPU times: user 21.1 ms, sys: 183 µs, total: 21.3 ms
Wall time: 517 ms


Das Model ist momentan noch verteilt, wir können trotzdem Predictions machen. Dafür müssen wir die Daten wieder verteilen, wenn diese nicht schon lokal vorliegen.

In [18]:
to_predict = X_test
print(f"Länge: {len(to_predict)} \nTyp: {type(to_predict)}\n {to_predict}")

Länge: 630 
Typ: <class 'cupy.ndarray'>
 [[  7.199896   -6.081043 ]
 [  8.88451    -4.293371 ]
 [-10.303796    7.9061947]
 ...
 [  7.657574   -3.5824952]
 [  7.6466017  -5.1047616]
 [  3.4916835  -6.8165197]]


Das cupy Array müssen wir in ein cuDF, dann in ein dask_cuDF umwandeln.

In [19]:
# to_predict__dask wird ein dask_cuDF
to_predict__dask = dask_cudf.from_cudf(cudf.DataFrame( to_predict ), npartitions=n_workers)

In [20]:
pred_task = multi_node_model.predict(to_predict__dask)
print(pred_task)

<dask_cudf.Series | 10 tasks | 3 npartitions>


Um die Berechnung zu starten schreiben wir `.compute()`

In [21]:
%time pred_futures__MultiNodeOrig_results = pred_task.compute()

CPU times: user 36.9 ms, sys: 1.03 ms, total: 38 ms
Wall time: 77 ms


In [22]:
# Index of table | result
pred_futures__MultiNodeOrig_results.head(3)

0    3
1    0
2    1
dtype: int32

# 3. Speichern, Laden, Testen

Das Speichern bei MNMG geht etwas anders, aber auch schnell und einfach. Dazu muss das verteilte Model zusammengefasst werden.

Multi-Node / Multi-GPU => Single-Node/Single-GPU Model

<br>

Speichern und Laden: https://docs.rapids.ai/api/cuml/stable/pickling_cuml_models/

In [23]:
import pickle

# Fasse Model zusammen
single_gpu_model = multi_node_model.get_combined_model()
# Speichern:
pickle.dump(single_gpu_model, open("template_data_Kmeans_Multi/cuml_MultieNode_kmeans_model.pkl", "wb"))

# Laden:
loaded_single_gpu_model = pickle.load(open("template_data_Kmeans_Multi/cuml_MultieNode_kmeans_model.pkl", "rb"))

Das Predicten sollte dann ohne Probleme funktionieren.

In [24]:
loaded_single_gpu_model.predict( cp.asarray( [ [1.5, 2.4] ], dtype=cp.float32 ))

0    2
dtype: int32

Wir können z.B. die trustworthiness Anwenden.

Wenn wir Sklearn für die trustworthiness nutzen wollen, muss der Typ des Inputs passen. Unsere Daten sind in dem Format cupy.ndarray.

Um Numpy zu nutzen, müssen wir nur  `.get()` hinter unsere Variablen schreiben, wie: `y.get()`. Oder wir wandeln die Daten vorher in ein Numpy um.

In [25]:
from sklearn.manifold import trustworthiness   # CPU Berechnung 
# Oder auch:
from cuml.metrics.trustworthiness import trustworthiness as cuml_trustworthiness   # GPU Berechnung, viel schneller

In [26]:
# cuML trustworthiness | kann lange dauern
X_embedded = single_gpu_model.transform(X_test)  
%time score = cuml_trustworthiness( X_test, X_embedded )
print(f"Score: {score}")

CPU times: user 3.5 ms, sys: 0 ns, total: 3.5 ms
Wall time: 3.41 ms
Score: 0.9983912621854744


Wir können wieder Predictions machen.

In [27]:
results_loadedModel = loaded_single_gpu_model.predict(X_test)
results_loadedModel.head(5)

0    3
1    0
2    1
3    7
4    3
dtype: int32

## 3.1 Multi-Node/-Multi-GPU Predictions

Das trainierte Model ist momentan nur für den Single-GPU Bereich einsetzbar. Um die Predictions zu verteilen kann man.:<br>
1) Jedem Worker das Model geben und die Daten hineinladen.
2) Das Model in ein MNMG Model überführen.


### 3.1.1 Erste Möglichkeit (bevorzugt):


Das gespeicherte Model befindet sich auf dem Host oder wurde schon verteilt. In dem Fall, dass sich das Model auf dem Host befindet, muss es an jeweilige Worker verteilt werden.

In [28]:
# Host lädt Model
loaded_single_gpu_model = pickle.load(open("template_data_Kmeans_Multi/cuml_MultieNode_kmeans_model.pkl", "rb"))
loaded_single_gpu_model

KMeansMG()

Mit Dask können wir das Model an alle Worker senden.

In [29]:
futures = client.scatter(loaded_single_gpu_model, broadcast=True)

In [30]:
# Future sollte den status finished angeben
futures

<Future: finished, type: cuml.cluster.kmeans_mg.KMeansMG, key: KMeansMG-1256ccceb6666ea9ea8b04dc55f96caf>

Jetzt können wir eine Funktion schreiben die uns die Predictions liefert. Wir können Lokale Daten nehmen oder erst wieder Daten für die Prediction verteilen.

In [31]:
# Verteile Daten, Nutze Dask-cuDF
to_predict__dask = dask_cudf.from_cudf(cudf.DataFrame( X_test ), npartitions=n_workers)  #n_workers
to_predict__dask

,0,1
npartitions=3,,
0,float32,float32
210,...,...
420,...,...
629,...,...


Wenn die Daten mit Dask-cuDF <u>vom Host erstellt werden</u>, müssen die Partitionen übergeben werden.

<img src="./pictures/pred1.PNG"  width="925px;" hight="925px;">

<img src="./pictures/pred2.PNG"  width="325px;" hight="225px;" >

Mit Dask Delayed sagen wir, dass diese Funktion parallel ausgeführt werden soll. In der Funktion können wir verteilt Daten lesen und bearbeiten, bevor wir die Predictions machen. 

<br>

Dask Delayed: https://docs.dask.org/en/stable/delayed.html

In [61]:
# Länge der Testdaten
len(X_test)

630

Wir haben die Daten als Dask-cuDF gelesen. Wenn wir das als Host machen, müssen wir in einer Schleife die Partitionen übergeben.

In [32]:
@dask.delayed
def dask_predict(test_data, part):  # Jeder Worker führt das aus. 
    
    # Das wird in der Console ausgegeben. Damit sehen wir das alles passt, und jeder worker seine Partitionen nimmt. 
    print(f"type: {type(test_data)} länge: {len(test_data)}\nHead:\n{test_data.head(2)}\nPartition: {part}")

    predictions = loaded_single_gpu_model.predict(test_data) 

    return predictions

Wir müssen die Anzahl der Partitionen angeben. Die Worker werden die eigenen Partitionen bearbeiten.

In [33]:
results = []
parts = 3   # Anzahl Partitionen

for part in range(parts):
    fut = dask_predict( to_predict__dask.get_partition(part), part )
    results.append( fut )
    
results = dask.compute(*results)

print(f"Type: {type(results)}\n")

Type: <class 'tuple'>



In [34]:
# Mache daraus ein Numpy Array.
deleayed_pred = []
for i in range( len(results) ):
    deleayed_pred = np.append(deleayed_pred, np.asarray( results[i].to_numpy()) )
type(deleayed_pred)

numpy.ndarray

Danach werden wir überprüfen, ob die gemachten Predictions gleich sind.

In [36]:
## Überprüfe ob das Array pred_futures__MultiNodeOrig_results == deleayed_pred ist.
# cp.asnumpy() formt das Array in Numpy um.
rapids_tools.arrays_equal( cp.asnumpy( pred_futures__MultiNodeOrig_results ), cp.asnumpy( results_loadedModel ) )

Gebe 10 Zeilen aus, die ungleich sind


Eine andere Möglichkeit ist, dass die Worker die Daten lokal lesen und ggf. auch für sich diese Partitionieren. 

Es können wieder veschiedene Quellen angegeben werden. Für ein einfaches kleines Beispiel, schreibt jeder Worker eine CSV Datei die später gelesen wird. 

In [85]:
# Diese Datei wird im lokalen Verzeichnis geschrieben.
np.savetxt('csv_file.csv', cp.asnumpy( X_test ), delimiter=",", header="A,B")

In [92]:
## Damit schreibt jeder Worker diese Datei.
@dask.delayed    # Jeder Worker führt das aus. 
def dask_write(test_data):
    print("Write file")
    np.savetxt('csv_file.csv', np.asarray(test_data, dtype=np.int32), delimiter=",")
    
futs = []
for i in range(3):
    futs.append(dask_write( cp.asnumpy( X_test ) ))
dask.compute(*futs)

(None, None, None)

In [87]:
# Wenn man das so macht, macht das nur ein Worker. 
fut = dask_write(cp.asnumpy(X_test))
fut.compute()

jetzt können wir diese CSV Datei laden und für das Testen nutzen. <br>
Am Ende bekommen wir wieder ein Tupel. Diesemal werden alle Daten genutzt.


In [114]:
@dask.delayed
def dask_predict_2():

    test_data = cudf.read_csv("csv_file.csv")       # Lese lokale Daten
    test_data = test_data.astype(dtype='float32')  # Muss float32 sein, oder ändere IO
    
    # oder- nutze Dask-cuDF um lokale Daten zu partitionieren. 
    #test_data = dask_cudf.read_csv("big_csv.csv",  blocksize="5KB")       # Lese lokale Daten, gebe Blocksize an
    #test_data = test_data.astype(dtype='float32')                         # Muss float32 sein, oder ändere IO
    
    print(f"Data loaded, len: {len(test_data)}")
    
    predictions = loaded_single_gpu_model.predict(test_data)
    return predictions

In [115]:
futs = []
for i in range(3):
    futs.append(dask_predict_2())
    

In [116]:
dask.compute(*futs)

(0      0
 1      1
 2      7
 3      3
 4      5
       ..
 624    1
 625    6
 626    0
 627    0
 628    4
 Length: 629, dtype: int32,
 0      0
 1      1
 2      7
 3      3
 4      5
       ..
 624    1
 625    6
 626    0
 627    0
 628    4
 Length: 629, dtype: int32,
 0      0
 1      1
 2      7
 3      3
 4      5
       ..
 624    1
 625    6
 626    0
 627    0
 628    4
 Length: 629, dtype: int32)

Man sieht das alle Worker alle Predictions gemacht haben. 

### 3.1.2 Zweite Möglichkeit 


Bei dieser Möglichkeit erstellen wir ein MNMG Model und übernehmen die Parameter des Single-Models. Es kann sein, dass es (noch) nicht bei jedem Algorithmus so geht. Es ist etwas tricky. Potenziell können nicht alle Methoden des Models erfolgreich ausgeführt werden. 

Welche Werte übergeben werden, ist in dem Modul rapids_tools.py unter "mnmg_kmeans__singleModel_to_multiModel()" zu sehen.

<u>Hinweis:</u><br>
Ob am Ende dieselben Ergebnisse herauskommen, sollte stückweise überprüft werden. Es könnten Abweichungen auftreten gegenüber dem Single-Node Model.

In [47]:
# Host lädt Model
loaded_single_gpu_model = pickle.load(open("template_data_Kmeans_Multi/cuml_MultieNode_kmeans_model.pkl", "rb"))
type(loaded_single_gpu_model)

cuml.cluster.kmeans_mg.KMeansMG

In [48]:
# Erstelle ein MNMG Basismodel. 
distributed_model = multi_kmeans(init="k-means||", random_state=100)

Danach nutzen wir eine Funktion um Parameter und Daten in das MNMG Model zu überführen.

In [49]:
distributed_model = rapids_tools.mnmg_kmeans__singleModel_to_multiModel(client, loaded_single_gpu_model, distributed_model)
distributed_model

K-Means: Überführe cuMl SNSG zu cuML MNMG.


Danach können wir wieder Daten lokal einlesen oder Daten erst verteilen, hier verteilen wir die Testdaten.

In [50]:
to_predict = X_test
print(f"Länge: {len(to_predict)} \nTyp: {type(to_predict)}\n {to_predict}")

Länge: 630 
Typ: <class 'cupy.ndarray'>
 [[  7.199896   -6.081043 ]
 [  8.88451    -4.293371 ]
 [-10.303796    7.9061947]
 ...
 [  7.657574   -3.5824952]
 [  7.6466017  -5.1047616]
 [  3.4916835  -6.8165197]]


In [51]:
to_predict = cp.asarray(to_predict)
to_predict__dask = dask_cudf.from_cudf(cudf.DataFrame( to_predict ), npartitions=3)
to_predict__dask

,0,1
npartitions=3,,
0,float32,float32
210,...,...
420,...,...
629,...,...


In [52]:
pred_futures = distributed_model.predict(to_predict__dask)
print(pred_futures)

<dask_cudf.Series | 10 tasks | 3 npartitions>


In [53]:
%time result_rebuild = pred_futures.compute()

CPU times: user 7.56 ms, sys: 5.08 ms, total: 12.6 ms
Wall time: 65.1 ms


In [54]:
result_rebuild.head(5)

0    3
1    0
2    1
3    7
4    3
dtype: int32

Jetzt können wir prüfen ob die gemachten Predictions gleich sind. Die untere Funktion vergleich ob die Zahlen identisch sind. 

In [55]:
# Predictions des trainierten Models was verteilt ist  vs  Das SNSG Model was wir in ein MNMG Model überführt haben.
# - Vorher muss es mit cp.asnumpy() in Numpy umgewandelt werden 
rapids_tools.arrays_equal( cp.asnumpy( pred_futures__MultiNodeOrig_results ), cp.asnumpy( result_rebuild ) )

Gebe 10 Zeilen aus, die ungleich sind


In [56]:
# Wenn etwas nicht stimmt, kommt so eine Ausgabe:
rapids_tools.arrays_equal([1, 2, 3], [1, 2, 4])

Gebe 10 Zeilen aus, die ungleich sind
X: 3 	!= 	 y: 4 


AssertionError: 
Items are not equal:
item=2

 ACTUAL: 3
 DESIRED: 4

In [57]:
rapids_tools.arrays_equal( cp.asnumpy( pred_futures__MultiNodeOrig_results ), cp.asnumpy( results_loadedModel ) )

Gebe 10 Zeilen aus, die ungleich sind


# 4. cuML Kmeans => Sklearn Kmeans Model 

In [59]:
from sklearn.cluster import KMeans as sk_Kmeans

Das Model bietet keine normale Überführung in ein CPU-Model. Es ist jedoch möglich, einige Werte dem Sklearn Model zu übergeben.

Ist das Ziel das Model später auf einem System zu nutzen, das keine CPU hat, kann das trainierte Model in ein Sklearn Model "konvertiert" werden.


In [60]:
# Erstelle Basismodel
sk_model_new =  sk_Kmeans()

In [61]:
sk_model_new  = rapids_tools.mnmg_kmeans__singleModel_to_sklearnModel(loaded_single_gpu_model, sk_model_new)

K-Means: Überführe cuML Model zu Sklearn Model.


Danach sollte eine Prediction möglich sein.

In [62]:
sk_model_new.predict(np.asarray( [ [1.5, 2.4] ], dtype=np.float32 ))

array([2], dtype=int32)

Dann können wir das Model speichern und wieder Laden.

In [63]:
# Speichern 
pickle.dump(sk_model_new, open("template_data_Kmeans_Multi/sklearn_kmeans_model.pkl", "wb"))
# Laden
loaded_sklearn_model = pickle.load(open("template_data_Kmeans_Multi/sklearn_kmeans_model.pkl", "rb"))

In [64]:
loaded_sklearn_model.predict(np.asarray( [ [1.5, 2.4] ], dtype=np.float32 ))

array([2], dtype=int32)

In [65]:
# cuML trustworthiness
X_embedded = loaded_sklearn_model.transform(X_test.get())  
%time score = cuml_trustworthiness(X_test, X_embedded )
print(f"Score: {score}")

CPU times: user 960 µs, sys: 248 µs, total: 1.21 ms
Wall time: 1.16 ms
Score: 0.9983912621854744


Wir können wieder die Predictions überprüfen. Der Input muss vom Typ Numpy sein.

In [66]:
sk_model_pred = loaded_sklearn_model.predict( cp.asnumpy(X_test) )

In [67]:
rapids_tools.arrays_equal( cp.asnumpy( results_loadedModel ), sk_model_pred )

Gebe 10 Zeilen aus, die ungleich sind


In [120]:
client.shutdown()